# Ultrasound Resolution Verification

## Configuration

### Library Path

Define where the library files (eg. `ornot.so` and `ogl_beamformer_lib.so` on Linux) can be found. If you are running this out of the ornot release folder you will not need to modify this.

In [ ]:
library_path = '..'

### Data Files

Provide the location of the offline data files to calculate resolution with.

In [ ]:
params_file = 'parameters.bp'
data_file   = 'data.zst'

## Beamforming Properties

Define the parameters you would like to use for creating the Point Spread Functions (PSFs). A 2D image will also be shown below so that you can verify if the location of the wire target is correct.

In [ ]:
wire_location_x   = -0.40e-3
wire_location_z   = 60.95e-3
region_width      = 4e-3
points            = 1024

## Beamform

For this part an instance of the Beamformer must be running (`ogl.exe`/`ogl`).

In [ ]:
from BeamformingRuns import BeamformingRuns
runs = BeamformingRuns(library_path)

output = runs.single_wire(parameters_file, data_file, wire_location_x, wire_location_z, region_width, points)

If you get an error you must ensure that the beamformer is running. You may need to restart the Jupyter kernel if you closed and reopened the beamformer.

## Output Plots

### 2D Display Parameters

`display_power_threshold` adjusts the apparent brightness of the displayed image. Smaller values are brighter and larger values are dimmer.

In [ ]:
display_power_threshold = 78

### 2D Wire Target

Here we want the wire to be close to the center of the image.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

threshold_value = np.pow(10.0, display_power_threshold / 20.0)
image = np.maximum(0, np.minimum(np.abs(output['image']), threshold_value))
image = image / threshold_value

fig, ax = plt.subplots(1, 1)
ax.imshow(image, extent=output['image_extent'] * 1e3, cmap='gray')
ax.set_ylabel("Z [mm]")
ax.set_xlabel("X [mm]")

### PSFs

Just like above we want the peak to be in the center of the axis.

In [ ]:
fig, axs = plt.subplots(1, 2)

lateral = np.abs(output['lateral'])
lateral = 20 * np.log10(lateral / lateral.max())
axial   = np.abs(output['axial'])
axial   = 20 * np.log10(axial / axial.max())

lateral_axis = output['lateral_axis'] * 1e3
axial_axis   = output['axial_axis']   * 1e3
axs[0].plot(lateral_axis, lateral)
axs[1].plot(axial_axis, axial)

axs[0].set_title("Lateral PSF")
axs[0].set_xlabel("X [mm]")
axs[0].set_ylabel("Magnitude [dB]")
axs[0].set_xlim((lateral_axis.min(), lateral_axis.max()))
axs[0].set_ylim((-30, 0))

axs[1].set_title("Axial PSF")
axs[1].set_xlabel("Z [mm]")
axs[1].set_ylabel("Magnitude [dB]")
axs[1].set_xlim((axial_axis.min(), axial_axis.max()))
axs[1].set_ylim((-30, 0))
axs[1].yaxis.set_label_position("right")
axs[1].yaxis.tick_right()

### Calculate Resolutions

In [ ]:
axial_span         = axial_axis[axial > -6]
lateral_span       = lateral_axis[lateral > -6]

axial_center       = axial_axis[axial.argmax()]
lateral_center     = lateral_axis[lateral.argmax()]

axial_resolution   = axial_span[-1]   - axial_span[0]
lateral_resolution = lateral_span[-1] - lateral_span[0]

print(f"Axial Resolution:    {axial_resolution:.3f} [mm]")
print(f"Lateral Resolution:  {lateral_resolution:.3f} [mm]")
print(f"Location of Maximum: <{lateral_center:.3f}, {axial_center:.3f}> [mm]")